In [3]:
import re

def remove_example_part(input_string):
    # 使用正则表达式来匹配并删除 "Below is an example..." 到 "(END OF EXAMPLES)" 的部分
    cleaned_string = re.sub(r'Below is an example of how to approach the task:.*?\(END OF EXAMPLES\)', '', input_string, flags=re.DOTALL)
    return cleaned_string

In [ ]:
import json

# 打开并读取 JSONL 文件
file_path = './CRS_scratchpad_20241223_203210.jsonl'  # 请替换为你的 JSONL 文件路径


result_list = []  # 用来存储所有的 input-output 对象

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # 将每行字符串解析为 Python 字典
            data = json.loads(line)
            if data["is_correct"]:
                # 提取 'scratchpad' 部分
                scratchpad = data.get('scratchpad', {})
                
                # 遍历 scratchpad 中的每个键值对
                for key, actions in scratchpad.items():
                    # 遍历每个 action
                    for action_key, action_data in actions.items():
                        if action_key.startswith('action '):
                            print(action_data)
                            input_data = action_data.get('input', 'No input available')
                            output_data = action_data.get('output', 'No output available')
                            
                            # 将 input 和 output 组成一个字典并添加到结果列表
                            result_list.append({
                                "instruction": remove_example_part(input_data),
                                "system": "You are a helpful assistant",
                                "input": "",
                                "output": output_data
                            })
                    
        except json.JSONDecodeError as e:
            print(f"解析错误：{e}")

# 输出最终的结果列表
#print(json.dumps(result_list, indent=4))  # 美化输出，查看整个列表

# 可选择保存到一个新的 JSON 文件
output_file = './amazon_game_sft.json'  # 请替换为你想保存的路径
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(result_list, outfile, ensure_ascii=False, indent=4)

